In [1]:
import os
import sys
import csv
import pickle
import random
import numpy as np
from time import time
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [2]:
def seed_everything(seed: int):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True
    
seed_everything(1234)

In [3]:
def read_data(path):
    with open(path, 'r') as csvfile:
        train_data = list(csv.reader(csvfile))[1:] # skip col name
        sents, lbls = [], []
        for s, l in train_data:
            sents.append(s)
            lbls.append(l)
    return sents, lbls

# number of parameters in model
def get_total_model_params(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

In [4]:
class CleavageDataset(Dataset):
    def __init__(self, seq, lbl):
        self.seq = seq
        self.lbl = lbl

    def __getitem__(self, idx):
        return self.lbl[idx], self.seq[idx]

    def __len__(self):
        return len(self.lbl)


class CleavageBatch:
    def __init__(self, batch):
        lbls, _, self.seq = tokenizer(batch)
        self.lbl = torch.tensor([int(l) for l in lbls], dtype=torch.float)

    def pin_memory(self):
        self.seq = self.seq.pin_memory()
        self.lbl = self.lbl.pin_memory()
        return self


def collate_wrapper(batch):
    return CleavageBatch(batch)

In [5]:
class ESM1b_BiLSTM(nn.Module):
    def __init__(self, esm1b, rnn_size, hidden_size, dropout):
        super().__init__()

        self.esm1b_encoder = esm1b

        self.dropout = nn.Dropout(dropout)

        self.lstm = nn.LSTM(
            input_size=1280,
            hidden_size=rnn_size,
            bidirectional=True,
            batch_first=True,
        )

        self.fc1 = nn.Linear(rnn_size * 2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, 1)

    def forward(self, seq):
        with torch.no_grad():
            # input shape: (batch_size, seq_len=10+2(cls, eos))
            # out: (batch_size, seq_len, embedding_dim=1280)
            result = self.esm1b_encoder(seq, repr_layers=[33])
        
        embedded = self.dropout(result['representations'][33][:, 1:10+1, :])

        # input shape: (batch_size, seq_len, embedding_dim)
        out, _ = self.lstm(embedded)

        # input shape: (batch_size, seq_len=1, 2*rnn_size)
        pooled, _ = torch.max(out, dim=1)

        # input shape: (batch_size, 2*rnn_size)
        out = self.dropout(F.relu(self.fc1(pooled)))

        # input shape: (batch_size, hidden_size)
        # output shape: (batch_size)
        return self.fc2(out).squeeze()

In [6]:
def process(model, loader, criterion, optim=None):
    epoch_loss, num_correct, total = 0, 0, 0
    
    for batch in tqdm(
        loader,
        desc="Train: " if optim is not None else "Eval: ",
        file=sys.stdout,
        unit="batches"
    ):
        seq, lbl = batch.seq, batch.lbl
        seq, lbl = seq.to(device), lbl.to(device)
        
        scores = model(seq)
        loss = criterion(scores, lbl)
        
        if optim is not None:
            optim.zero_grad()
            loss.backward()
            optim.step()
        
        epoch_loss += loss.item()
        num_correct += ((scores > 0) == lbl).sum()
        total += len(seq)
    return epoch_loss / total, num_correct / total

In [7]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# load pre-trained esm1b model and vocab
esm1b, vocab = torch.hub.load('facebookresearch/esm:main', 'esm1b_t33_650M_UR50S')

# load train and dev data
train_seqs, train_lbl = read_data("../../data/n_train.csv")
dev_seqs, dev_lbl = read_data("../../data/n_val.csv")

tokenizer = vocab.get_batch_converter()

Using cache found in /home/ingo/.cache/torch/hub/facebookresearch_esm_main


In [8]:
NUM_EPOCHS = 10
BATCH_SIZE = 512
RNN_SIZE = 512
HIDDEN_SIZE = 128
DROPOUT = 0.5
LEARNING_RATE = 1e-4

model = ESM1b_BiLSTM(
    esm1b=esm1b,
    rnn_size=RNN_SIZE,
    hidden_size=HIDDEN_SIZE,
    dropout=DROPOUT
).to(device)

optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)
criterion = nn.BCEWithLogitsLoss()

# create train and dev loader
train_data = CleavageDataset(train_seqs, train_lbl)
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

dev_data = CleavageDataset(dev_seqs, dev_lbl)
dev_loader = DataLoader(dev_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_wrapper, pin_memory=True, num_workers=10)

print(f"Total model parameters: {get_total_model_params(model):,}")

Total model parameters: 659,838,647


In [9]:
start = time()
print("Starting Training.")
highest_val_acc = 0
train_losses, train_accuracies= [], []
val_losses, val_accuracies = [], []

for epoch in range(1, NUM_EPOCHS + 1):
    model.train()
    train_loss, train_acc = process(model, train_loader, criterion, optimizer)
    
    model.eval()
    with torch.no_grad():
        val_loss, val_acc = process(model, dev_loader, criterion)
        
    # save current acc, loss
    train_losses.append((epoch, train_loss))
    train_accuracies.append((epoch, train_acc))
    val_losses.append((epoch, val_loss))
    val_accuracies.append((epoch, val_acc))
    
    if val_acc > highest_val_acc:
        highest_val_acc = val_acc
        path = f"../../params/n_term/esm1b/acc{val_acc:.4f}_epoch{epoch}.pt"
        torch.save(model.state_dict(), path)
        
    print(
        f"Training:   [Epoch {epoch:2d}, Loss: {train_loss:8.4f}, Acc: {train_acc:.4f}]"
    )
    print(f"Evaluation: [Epoch {epoch:2d}, Loss: {val_loss:8.4f}, Acc: {val_acc:.4f}]")
    
print("Finished Training.")
train_time = (time() - start) / 60
print(f"Training took {train_time} minutes.")

Starting Training.
Eval: 100%|███████████████████████████████████████████████████████| 280/280 [01:36<00:00,  2.89batches/s]
Training:   [Epoch  1, Loss:   0.0012, Acc: 0.6346]
Evaluation: [Epoch  1, Loss:   0.0011, Acc: 0.6868]
Eval: 100%|███████████████████████████████████████████████████████| 280/280 [01:37<00:00,  2.89batches/s]
Training:   [Epoch  2, Loss:   0.0011, Acc: 0.6840]
Evaluation: [Epoch  2, Loss:   0.0011, Acc: 0.6883]
Eval: 100%|███████████████████████████████████████████████████████| 280/280 [01:37<00:00,  2.89batches/s]
Training:   [Epoch  3, Loss:   0.0011, Acc: 0.6875]
Evaluation: [Epoch  3, Loss:   0.0011, Acc: 0.6915]
Eval: 100%|███████████████████████████████████████████████████████| 280/280 [01:37<00:00,  2.89batches/s]
Training:   [Epoch  4, Loss:   0.0011, Acc: 0.6903]
Evaluation: [Epoch  4, Loss:   0.0011, Acc: 0.6941]
Eval: 100%|███████████████████████████████████████████████████████| 280/280 [01:37<00:00,  2.89batches/s]
Training:   [Epoch  5, Loss:   0.00

In [ ]:
# save training stats
lsts = [train_losses, train_accuracies, val_losses, val_accuracies, train_time]
names = [
    "train_losses",
    "train_accuracies",
    "val_losses",
    "val_accuracies",
    "train_time",
]
to_save = dict()
for name, lst in zip(names, lsts):
    to_save[name] = lst

with open(f"../params/n_term/quadBiLSTM/metrics.pkl", "wb") as f:
    pickle.dump(to_save, f, pickle.HIGHEST_PROTOCOL)

print("Finished Saving Details.")